# Load the data

In [3]:
import pandas as pd
in_datafile = "in/ttj.csv"
df = pd.read_csv(in_datafile)
df.head()

,parton_t_lep_Pt,parton_t_lep_Eta,parton_t_lep_Phi,parton_t_lep_M,parton_t_had_Pt,parton_t_had_Eta,parton_t_had_Phi,parton_t_had_M,parton_W_had_Pt,parton_W_had_Eta,...,b_lep_Phi,b_lep_E,b_had_Pt,b_had_Eta,b_had_Phi,b_had_E,W_lep_Pt,W_lep_Eta,W_lep_Phi,W_lep_E
0,26210.712891,-2.813584,1.222510,172521.125000,72533.125000,-3.675916,1.456350,172723.531250,114015.984375,-3.021512,...,2.412543,130343.023438,201853.781250,0.592402,-0.441704,238950.625000,135686.473118,2.138249,2.359626,589988.274388
1,92752.843750,-1.496029,2.613532,176464.171875,96631.039062,-2.584467,-0.316401,170910.125000,75805.507812,-2.085379,...,2.573960,92072.093750,34999.722656,-0.036871,-0.790957,35248.691406,36329.159212,1.782928,-1.464808,137125.985111
2,139194.140625,-0.391737,-3.099086,170435.546875,135427.390625,0.811365,0.089600,173165.109375,149896.234375,0.775999,...,-2.870163,104799.687500,41540.578125,2.103381,1.266825,172784.843750,82647.169437,-0.580798,-0.677408,125975.005508
3,35275.816406,1.068851,1.896412,172743.640625,44878.582031,1.920528,-2.050357,172766.781250,65037.789062,0.261693,...,-1.561372,270595.156250,27744.072266,0.817976,2.255633,37834.363281,102641.485570,2.250263,-1.864985,498973.213324
4,38095.621094,0.194375,-0.120800,171511.437500,83400.218750,-0.696042,-2.875081,173112.609375,117186.085937,-0.156181,...,-2.325078,216230.812500,59974.976562,-0.334010,1.645467,63570.820312,110140.405600,-1.242552,-0.792195,221770.231981


In [5]:
fraction_train = 0.65 
max_size = len(df)
train_df = df[:int(max_size*fraction_train)]
test_df  = df[int(max_size*fraction_train):max_size]
print "Created train dataframe of size: ", len(train_df)
print "Created test dataframe of size:  ", len(test_df)

Created train dataframe of size:  722
Created test dataframe of size:   390


### Create input X and Y vectors

Set-up the input and output vectors. 

The output vector is going to be the jet indices:
    ( 
    
        is_fisrt_bjet_leptonic
        is_second_bjet_hadronic,
        is_in_W
        is_in_W,
        is_in_W,
    )
Then some spectator variables needed for the custom loss function that minimizes differences in four momenta: 
    (
    
        parton_top_had_p4,
        parton_top_lep_p4,
        parton_w_had_p4,
        reco_w_lep_p4,
        reco_bjet_1
        reco_bjet_2
        reco_jet_1
        reco_jet_2
        reco_jet_3     
    )
    

In [6]:
import numpy as np

all_variables = list(df.columns.values)
reco_variables = all_variables[12:24]+all_variables[32:40]
# parton_variables = all_variables[0:12]+all_variables[24:32]+all_variables[40:44]
parton_variables = all_variables[0:12]+all_variables[40:44]

X_train = np.array(train_df[reco_variables])
Y_train = np.array(train_df[parton_variables])
for i in xrange(5):
    Y_train =  np.insert(Y_train, 0,0, axis=1)

print X_train.shape
print Y_train.shape

print parton_variables
print reco_variables

(722, 20)
(722, 21)
['parton_t_lep_Pt', ' parton_t_lep_Eta', ' parton_t_lep_Phi', ' parton_t_lep_M', ' parton_t_had_Pt', ' parton_t_had_Eta', ' parton_t_had_Phi', ' parton_t_had_M', ' parton_W_had_Pt', ' parton_W_had_Eta', ' parton_W_had_Phi', ' parton_W_had_M', ' W_lep_Pt', ' W_lep_Eta', ' W_lep_Phi', ' W_lep_E']
[' jet1_Pt', ' jet1_Eta', ' jet1_Phi', ' jet1_E', ' jet2_Pt', ' jet2_Eta', ' jet2_Phi', ' jet2_E', ' jet3_Pt', ' jet3_Eta', ' jet3_Phi', ' jet3_E', ' b_lep_Pt', ' b_lep_Eta', ' b_lep_Phi', ' b_lep_E', ' b_had_Pt', ' b_had_Eta', ' b_had_Phi', ' b_had_E']


# Basic plots

# Set-up the model

In [15]:
b = [1,2,3,4,5,6,7,8,9,10,11,12]
print b[:4]
print b[4:8]

[1, 2, 3, 4]
[5, 6, 7, 8]


In [23]:
import ROOT as r 
from keras import backend as K

def transform_to_jet_assignment(y_pred):
    """
        Convert a 3-vector that sums to unity into a vector of 3 predictions
        where:
            first element represents  if jet_1 is in the  hadronic W
            second element represents if jet_2 is in the  hadronic W
            third element represents  if bjet_1 is in the hadronic top
            
            The jet_3 is therefore in the hadronic_W when (2 - [0]-[1] ) == 1
            The bjet_2 is therefore in the hadronic_W when (1 - [2] ) == 1
    """
    return y_pred

def mass_diff_loss(y_true,y_pred):
    """
        user defined evaluation function, return a pair (metric_name, result)        
        See the following links for details 
            https://github.com/keras-team/keras/issues/4781
            https://stackoverflow.com/questions/45961428/make-a-custom-loss-function-in-keras
        
        
        Intended to be used with 16 so-called spectator variables that correspond to
        the truth level leptonic and hadronic top, truth level hadronic W, and reconstucted
        leptonic W boson four momemtum
        
        A further 20 spectator variables are passed that correspond to the reconstruction level
        jets, 3 light jets and 2 b-jets. 
        
        The only part of y_pred that effects this loss function is the last 3 elements. These 
        correspond to jet assignment. 
    """
    
    # Note everything is done by tensor algebra - so this actually working things out for
    # every event in a batch all in one go using a highly optimized tensorflow backend
        
    # Transform predictions into booleans that represent
    # if a jet is a decay product of the hadronic top quark 
    assignment = transform_to_jet_assignment(y_pred[-3:]) 
    
    # Get the four vectors for each of the needed physics objects
    # we're in a basis such that each four vector = (px,py,pz,E)
    jet_1       = y_pred[0:4]
    jet_2       = y_pred[4:8]
    jet_3       = y_pred[8:12]
    bjet_1      = y_pred[12:16]
    bjet_2      = y_pred[16:20]
    w_lep_truth  = y_pred[20:24]
    t_lep_truth  = y_pred[24:28]
    t_had_truth  = y_pred[28:32]
    w_had_truth  = y_pred[32:36]
    
    # calculated the masses for all of the predictions and truth level objects
    top_hadronic_px  = assignment[0]*jet_1[0] + assignment[1]*jet_2[0] + (2-assignment[1])*jet_3[0] + assignment[2]*bjet_1[0]+ (1-assignment[2])*bjet_2[0]
    top_hadronic_py  = assignment[0]*jet_1[1] + assignment[1]*jet_2[1] + (2-assignment[1])*jet_3[1] + assignment[2]*bjet_1[1]+ (1-assignment[2])*bjet_2[1]
    top_hadronic_pz  = assignment[0]*jet_1[2] + assignment[1]*jet_2[2] + (2-assignment[1])*jet_3[2] + assignment[2]*bjet_1[2]+ (1-assignment[2])*bjet_2[2]
    top_hadronic_e   = assignment[0]*jet_1[3] + assignment[1]*jet_2[3] + (2-assignment[1])*jet_3[3] + assignment[2]*bjet_1[3]+ (1-assignment[2])*bjet_2[3]
    top_hadronic_m    = K.square(top_hadronic_e) - (K.square(top_hadronic_px)+K.square(top_hadronic_py)+K.square(top_hadronic_pz))
    
    w_hadronic_px  = assignment[0]*jet_1[0] + assignment[1]*jet_2[0]  + (2-assignment[1])*jet_3[0]
    w_hadronic_py  = assignment[0]*jet_1[1] + assignment[1]*jet_2[1]  + (2-assignment[1])*jet_3[1]
    w_hadronic_pz  = assignment[0]*jet_1[2] + assignment[1]*jet_2[2]  + (2-assignment[1])*jet_3[2]
    w_hadronic_e   = assignment[0]*jet_1[3] + assignment[1]*jet_2[3]  + (2-assignment[1])*jet_3[3]
    w_hadronic_m    = K.square(w_hadronic_e) - (K.square(w_hadronic_px)+K.square(w_hadronic_py)+K.square(w_hadronic_pz))
    
    t_lep_truth_m   = K.square(t_lep_truth[4]) - K.square(t_lep_truth[0]) - K.square(t_lep_truth[1]) - K.square(t_lep_truth[2])
    t_had_truth_m   = K.square(t_had_truth[4]) - K.square(t_had_truth[0]) - K.square(t_had_truth[1]) - K.square(t_had_truth[2])
    w_had_truth_m   = K.square(w_had_truth[4]) - K.square(w_had_truth[0]) - K.square(w_had_truth[1]) - K.square(w_had_truth[2])
    w_lep_truth_m   = K.square(w_lep_truth[4]) - K.square(w_lep_truth[0]) - K.square(w_lep_truth[1]) - K.square(w_lep_truth[2])
    
    top_lep_pz = w_lep_truth[0] + (1-assignment[2])*bjet_1[0]+ (assignment[2])*bjet_2[0]
    top_lep_py = w_lep_truth[1] + (1-assignment[2])*bjet_1[1]+ (assignment[2])*bjet_2[1]
    top_lep_px = w_lep_truth[2] + (1-assignment[2])*bjet_1[2]+ (assignment[2])*bjet_2[2]
    top_lep_e  = w_lep_truth[3] + (1-assignment[2])*bjet_1[3]+ (assignment[2])*bjet_2[3]
    top_lep_m   = K.square(top_lep_e) - (K.square(top_lep_px)+K.square(top_lep_py)+K.square(top_lep_pz))
    
    # Return the loss as the differences in mass for each of the reconstructed objects 
    return K.square(top_lep_m-t_lep_truth_m) + K.square(top_hadronic_m-t_had_truth_m)+K.square(w_hadronic_m-w_had_truth_m)


In [109]:
from keras.layers import Input, Dense
from keras.models import Model

# Construct the jet assignment part of the network, a simple MLP
jet_assignment_input  = Input(shape=(32,), dtype='float32', name='RecoJetInput')
jet_assignment_output = Dense(32, activation='relu', name='JetAssignmentLayer')(jet_assignment_input)
jet_assignment_output = Dense(3,  activation='relu', name='JetAssignment2ndLayer')(jet_assignment_input)

# Potenttially might want to add additional W mass reconstruction part of the network here??
# auxiliary_output = Dense(3, activation='sigmoid', name='aux_output')(jet_assignment_input)

# Merge in truth level information required for the 
auxiliary_input = Input(shape=(36,), name='TruthLevelInput')
x = concatenate([jet_assignment_output, auxiliary_input])

# And finally we add the main logistic regression layer
# output = Dense(3, activation='sigmoid', name='Output')(x)

model = Model(inputs=[auxiliary_input,jet_assignment_input], outputs=[x])
model.compile(optimizer='rmsprop', loss=mass_diff_loss)
model.summary()
plot_model(model, to_file='model.png',show_shapes=True)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
RecoJetInput (InputLayer)        (None, 32)            0                                            
____________________________________________________________________________________________________
JetAssignment2ndLayer (Dense)    (None, 3)             99          RecoJetInput[0][0]               
____________________________________________________________________________________________________
TruthLevelInput (InputLayer)     (None, 36)            0                                            
____________________________________________________________________________________________________
concatenate_58 (Concatenate)     (None, 39)            0           JetAssignment2ndLayer[0][0]      
                                                                   TruthLevelInput[0][0]   

# Train the model

In [ ]:
=